In [3]:
import pandas as pd
data=pd.read_csv("/content/placed.csv")
data

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot
0,22,Male,Electronics And Communication,1,8,1,1,1
1,21,Female,Computer Science,0,7,1,1,1
2,22,Female,Information Technology,1,6,0,0,1
3,21,Male,Information Technology,0,8,0,1,1
4,22,Male,Mechanical,0,8,1,0,1
...,...,...,...,...,...,...,...,...
2961,23,Male,Information Technology,0,7,0,0,0
2962,23,Male,Mechanical,1,7,1,0,0
2963,22,Male,Information Technology,1,7,0,0,0
2964,22,Male,Computer Science,1,7,0,0,0


In [4]:
print(data.isnull().sum())

Age                  0
Gender               0
Stream               0
Internships          0
CGPA                 0
Hostel               0
HistoryOfBacklogs    0
PlacedOrNot          0
dtype: int64


In [5]:
data.columns

Index(['Age', 'Gender', 'Stream', 'Internships', 'CGPA', 'Hostel',
       'HistoryOfBacklogs', 'PlacedOrNot'],
      dtype='object')

In [6]:
x=data[['Age', 'Stream', 'Internships', 'CGPA',
       'HistoryOfBacklogs']]
print(x)

      Age                         Stream  Internships  CGPA  HistoryOfBacklogs
0      22  Electronics And Communication            1     8                  1
1      21               Computer Science            0     7                  1
2      22         Information Technology            1     6                  0
3      21         Information Technology            0     8                  1
4      22                     Mechanical            0     8                  0
...   ...                            ...          ...   ...                ...
2961   23         Information Technology            0     7                  0
2962   23                     Mechanical            1     7                  0
2963   22         Information Technology            1     7                  0
2964   22               Computer Science            1     7                  0
2965   23                          Civil            0     8                  0

[2966 rows x 5 columns]


In [7]:
y=data[['PlacedOrNot']]
print(y)

      PlacedOrNot
0               1
1               1
2               1
3               1
4               1
...           ...
2961            0
2962            0
2963            0
2964            0
2965            1

[2966 rows x 1 columns]


In [8]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)
print(y)

[1 1 1 ... 0 0 1]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
le=LabelEncoder()
x['Stream'] = le.fit_transform(x['Stream'])
print(x['Stream'])

0       3
1       1
2       4
3       4
4       5
       ..
2961    4
2962    5
2963    4
2964    1
2965    0
Name: Stream, Length: 2966, dtype: int64


<ipython-input-11-95667ee9b98f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Stream'] = le.fit_transform(x['Stream'])


In [12]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x=scaler.fit_transform(x)

In [14]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
y_encoded=to_categorical(y)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y_encoded,test_size=0.2,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((2372, 5), (594, 5), (2372, 2), (594, 2))

In [15]:
from keras.models import Sequential
from keras.layers import Dense
model=Sequential()
model.add(Dense(32,input_shape=(X_train.shape[1],),activation='relu'))
#model.add(Dense(1,activation='relu'))
model.add(Dense(2,activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 258 (1.01 KB)

 Trainable params: 258 (1.01 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=50,batch_size=20,validation_data=(X_test,y_test))

Epoch 1/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8516 - loss: 0.3248 - val_accuracy: 0.8704 - val_loss: 0.3291
Epoch 2/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8638 - loss: 0.3057 - val_accuracy: 0.8721 - val_loss: 0.3275
Epoch 3/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8625 - loss: 0.3004 - val_accuracy: 0.8754 - val_loss: 0.3266
Epoch 4/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8598 - loss: 0.3159 - val_accuracy: 0.8653 - val_loss: 0.3259
Epoch 5/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8734 - loss: 0.2949 - val_accuracy: 0.8670 - val_loss: 0.3254
Epoch 6/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8713 - loss: 0.2962 - val_accuracy: 0.8670 - val_loss: 0.3249
Epoch 7/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8797 - loss: 0.2829 - val_accuracy: 0.8653 - val_loss: 0.3229
Epoch 8/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8600 - loss: 0.3106 - val_accuracy: 0.

In [22]:
model.evaluate(X_test,y_test)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8830 - loss: 0.2943 


[0.30587878823280334, 0.8771044015884399]

In [23]:
print('loss:',model.evaluate(X_test,y_test)[0])
print('accuracy:',model.evaluate(X_test,y_test)[1])

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8830 - loss: 0.2943 
loss: 0.30587878823280334
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8830 - loss: 0.2943 
accuracy: 0.8771044015884399
